In [2]:
using JuMP
using GLPK

# Exercise 1 
Consider the following table indicating the nutritional value of different food types:

|Foods|Price($) per serving|Calories per serving|Fat(g) per serving|Protein(g) per serving|Carbohydrate(g) per serving|
|---|---|---|---|---|---|
|Raw carrots|0.14|23|0.1|0.6|6|
|Baked potatoes|0.12|171|0.2|3.7|30|
|Wheat bread|0.2|65|0|2.2|13|
|Cheddar cheese|0.75|112|9.3|7|0|
|Peanut butter|0.15|188|16|7.7|2|

You need to decide how many servings of each food to buy each day so that you minimize the total cost of buying food while satisfying the following daily nutritional requirements:
-  Calories >= 2000 
-  Fat >= 50g
-  Protein >= 100g
-  Carbohydrates >= 250g

Formulating as a linear programming problem: let $x_1, x_2, x_3, x_4, x_5$ be the number of servings of each food in a ration. \
Then: \
min z(x) =    $0.14x_1 + 0.12x_2 + 0.2x_3 + 0.75x_4 + 0.15x_5$                     (0) - minimizing overall cost \
                     $23x_1 + 171x_2 + 65x_3 + 112x_4 + 188x_5 >= 2000$           (1) - requirement for calories \
                     $0.1x_1 + 0.2x_2 + 0x_3 + 9.3x_4 + 16x_5 >= 50$                    (2) - requirement for fats \
                     $0.6x_1 + 3.7x_2 + 2.2x_3 + 7x_4 + 7.7x_5 >= 100$                (3) - requirement for proteins \
                     $6x_1 + 30x_2 + 13x_3 + 0x_4 + 2x_5 >= 250$                         (4) - requirement for carbohydrates \
                     $x_1, x_2, x_3, x_4, x_5 >= 0$                                                          (5) - investment can be made or not and only once

Let's make an implicit model this time as there are more variables. \
c = [0.14, 0.12, 0.2, 0.75, 0.15] \
d = [2000, 50, 100, 250] \
T = [23 171 65 112 188 ; 0.1 0.2 0 9.3 16 ; 0.6 3.7 2.2 7 7.7 ; 6 30 13 0 2]

In [5]:
c = [0.14, 0.12, 0.2, 0.75, 0.15]
d = [2000, 50, 100, 250]
T = [23 171 65 112 188 ; 0.1 0.2 0 9.3 16 ; 0.6 3.7 2.2 7 7.7 ; 6 30 13 0 2]
n, m = size(T)

(4, 5)

In [7]:
model = Model(GLPK.Optimizer)
@variable(model, x[1:m] >= 0)
@objective(model, Min, sum(c[j]*x[j] for j=1:m))
@constraint(model, cst[i=1:n], sum(T[i,j]*x[j] for j=1:m) >= d[i])

4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cst[1] : 23 x[1] + 171 x[2] + 65 x[3] + 112 x[4] + 188 x[5] ≥ 2000
 cst[2] : 0.1 x[1] + 0.2 x[2] + 9.3 x[4] + 16 x[5] ≥ 50
 cst[3] : 0.6 x[1] + 3.7 x[2] + 2.2 x[3] + 7 x[4] + 7.7 x[5] ≥ 100
 cst[4] : 6 x[1] + 30 x[2] + 13 x[3] + 2 x[5] ≥ 250

In [8]:
print(model)

In [9]:
optimize!(model)

In [10]:
@show termination_status(model)
@show primal_status(model)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT


FEASIBLE_POINT::ResultStatusCode = 1

In [11]:
@show objective_value(model)
@show value.(x)

objective_value(model) = 2.317754919499105
value.(x) = [0.0, 7.714669051878354, 0.0, 0.0, 9.279964221824686]


5-element Vector{Float64}:
 0.0
 7.714669051878354
 0.0
 0.0
 9.279964221824686

In [12]:
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 2.31775e+00
  Objective bound    : -Inf
  Dual objective value : 2.31775e+00

* Work counters
  Solve time (sec)   : 6.56128e-04


## Answer 
To meet all the nutritional requirements at the minimal costs, it's recommended to buy each day 7.7 servings of baked potatoes and 9.3 servings of peanut butter. \
It will cost 2.32 $ (which is still not healthy at all)

# Exercise 2
MUCOW (Milk Undertakings, C and O, Waterloo) owns a herd of Holstein cows and a herd of Jersey cows. For each herd, the total number of litres produced each day, and milk-fat content (as a percentage) are as follows:

||Litres produced|Percent milk-fat|
|---|---|---|
|Holstein|500|3.7|
|Jersey|250|4.9|


The fat is split off and blended again to create various products. For each product, the volume, required milk-fat percentage, and profit are as follows. In particular, the milk-fat percentage must exactly equal what is specified.

||Skimmed milk|2%|Whole milk|Table cream|Whipping cream|
|---|---|---|---|---|---|
|Volume(litres)|2|2|2|0.6|0.3|
|Milk-fat requirement|0%|2%|4%|15%|45%|
|Profit($)|0.1|0.15|0.2|0.5|1.2|

(a) Formulate as an LP the problem of deciding how many items of each product to produce to maximize the total profit. \
Formulating as a linear programming problem: let $x_1, x_2, x_3, x_4, x_5$ be the number of each product to be produced. \
Then: \
max z(x) =    $0.1x_1 + 0.15x_2 + 0.2x_3 + 0.5x_4 + 1.2x_5$                                                                                                                                (0) - maximizing overall profit \
                     $2x_1 + 2x_2 + 2x_3 + 0.6x_4 + 0.3x_5 <= 750$                                                                                                                              (1) - constraint for volume \
                     $(0*2)x_1 + (0.02*2)x_2 + (0.04*2)x_3 + (0.15*0.6)x_4 + (0.45*0.3)x_5 <= 500*0.037 + 250*0.049$              (2) - constraint for available fats \
                     $x_1, x_2, x_3, x_4, x_5 >= 0$                                                                                                                                                                 (3) - number items can't be negative \
In matrix notation:

      (2    2      2     0.6    0.3)\
A = 0 0.04 0.08 0.09 0.135 

b = (750 30.75)

c = (0.1 0.15 0.2 0.5 1.2)
                     

In [23]:
c = [0.1, 0.15, 0.2, 0.5, 1.2]
d = [750 30.75]
T = [2 2 2 0.6 0.3 ;0 0.04 0.08 0.09 0.135]
n, m = size(T)

(2, 5)

In [15]:
model2 = Model(GLPK.Optimizer)
@variable(model2, x[1:m] >= 0)
@objective(model2, Max, sum(c[j]*x[j] for j=1:m))
@constraint(model2, cst[i=1:n], sum(T[i,j]*x[j] for j=1:m) <= d[i])

2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cst[1] : 2 x[1] + 2 x[2] + 2 x[3] + 0.6 x[4] + 0.3 x[5] ≤ 750
 cst[2] : 0.04 x[2] + 0.08 x[3] + 0.09 x[4] + 0.135 x[5] ≤ 30.75

In [16]:
print(model2)

In [17]:
optimize!(model2)

In [18]:
@show termination_status(model2)
@show primal_status(model2)

termination_status(model2) = MathOptInterface.OPTIMAL
primal_status(model2) = MathOptInterface.FEASIBLE_POINT


FEASIBLE_POINT::ResultStatusCode = 1

In [19]:
@show objective_value(model2)
@show value.(x)

objective_value(model2) = 307.41666666666663
value.(x) = [340.8333333333333, 0.0, 0.0, 0.0, 227.77777777777777]


5-element Vector{Float64}:
 340.8333333333333
   0.0
   0.0
   0.0
 227.77777777777777

In [21]:
solution_summary(model2)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 3.07417e+02
  Objective bound    : Inf
  Dual objective value : 3.07417e+02

* Work counters
  Solve time (sec)   : 2.19345e-05


(b) MUCOW is told of a regulation change: 'skimmed milk' can now contain anything up to 0.1% milk-fat, but no more. How does the formulation of the problem change? 

The change in the formula will be only in constraint for available fet for skimmed milk

In [26]:
c = [0.1, 0.15, 0.2, 0.5, 1.2]
d = [750 30.75]
T = [2 2 2 0.6 0.3 ;0.01 0.04 0.08 0.09 0.135]
n, m = size(T)

(2, 5)

In [27]:
model3 = Model(GLPK.Optimizer)
@variable(model3, x[1:m] >= 0)
@objective(model3, Max, sum(c[j]*x[j] for j=1:m))
@constraint(model3, cst[i=1:n], sum(T[i,j]*x[j] for j=1:m) <= d[i])

2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cst[1] : 2 x[1] + 2 x[2] + 2 x[3] + 0.6 x[4] + 0.3 x[5] ≤ 750
 cst[2] : 0.01 x[1] + 0.04 x[2] + 0.08 x[3] + 0.09 x[4] + 0.135 x[5] ≤ 30.75

In [28]:
print(model3)

In [29]:
optimize!(model3)

In [30]:
@show objective_value(model3)
@show value.(x)

objective_value(model3) = 277.16292134831457
value.(x) = [344.66292134831457, 0.0, 0.0, 0.0, 202.24719101123594]


5-element Vector{Float64}:
 344.66292134831457
   0.0
   0.0
   0.0
 202.24719101123594

In [31]:
solution_summary(model3)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 2.77163e+02
  Objective bound    : Inf
  Dual objective value : 2.77163e+02

* Work counters
  Solve time (sec)   : 5.91278e-05


## Answer
(a) The optimal number of items to produce is 340.8 items of skimmed milk and 227.8 items of whipped cream.\
Total revenue will be $307.4

(b) The optimal number of items to produce is 344.7 items of skimmed milk and 202.2 items of whipped cream.\
Total revenue will be $277.2

# Exercise 3

You are about to trek across the desert with a vehicle having 3.6 cubic metres of cargo space for goods. There are various types of items available for putting in this space, each with a different volume and a different net value for your trip, shown as follows:

|Item type i|1|2|3|4|5|6|7|
|---|---|---|---|---|---|---|---|
|Volume $v_i$|0.55|0.6|0.7|0.75|0.85|0.9|0.95|
|Net value $n_i$|250|300|500|700|750|900|950|

(a) You need to decide which items to take, not exceeding the volume constraint. You can take at most one of any item. No item can be split into fractions. The total net value must be maximized. Formulate this problem as an LP or IP.

Formulating as a LP problem: let $x_1, x_2, x_3, x_4, x_5, x_6, x_7$ be the indicatiors if the item should be taken or not. \
Then: \
max z(x) =    $250x_1 + 300x_2 + 500x_3 + 700x_4 + 750x_5 + 900x_6 + 950x_7$                                            (0) - maximizing overall profit \
                     $0.55x_1 + 0.6x_2 + 0.7x_3 + 0.75x_4 + 0.85x_5 + 0.9x_6 + 0.95x_7 <= 3.6$                                   (1) - constraint for volume \
                     $x_1, x_2, x_3, x_4, x_5, x_6, x_7 \in \{0; 1\}$                                                                           (3) - number items can't be negative \

In [34]:
c = [250, 300, 500, 700, 750, 900, 950]
d = [3.6]
T = [0.55 0.6 0.7 0.75 0.85 0.9 0.95]
n, m = size(T)

(1, 7)

In [35]:
model4 = Model(GLPK.Optimizer)
@variable(model4, x[1:m], Bin)
@objective(model4, Max, sum(c[j]*x[j] for j=1:m))
@constraint(model4, cst[i=1:n], sum(T[i,j]*x[j] for j=1:m) <= d[i])

1-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cst[1] : 0.55 x[1] + 0.6 x[2] + 0.7 x[3] + 0.75 x[4] + 0.85 x[5] + 0.9 x[6] + 0.95 x[7] ≤ 3.6

In [36]:
print(model4)

In [37]:
optimize!(model4)

In [38]:
@show termination_status(model4)
@show primal_status(model4)

termination_status(model4) = MathOptInterface.OPTIMAL
primal_status(model4) = MathOptInterface.FEASIBLE_POINT


FEASIBLE_POINT::ResultStatusCode = 1

In [39]:
@show objective_value(model4)
@show value.(x)

objective_value(model4) = 3300.0
value.(x) = [0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]


7-element Vector{Float64}:
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0

In [40]:
solution_summary(model4)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 3.30000e+03
  Objective bound    : 3.30000e+03
  Relative gap       : 1.51515e-02

* Work counters
  Solve time (sec)   : 4.14991e-03


(b) Your two friends have decided to come as well, each with an identical vehicle. There are exactly two items of each type. The question is, can you fit all 14 items in the vehicles without exceeding the volume comstraints? No cutting items into pieces is permitted! Each item taken must be placed entirely in one of the vehicles. Net value is ignored for this part.

In this case we will be moving from the point, that we are solving bin packing problem and trying to assign all items into 3 vehicles.\
If the solution will be feasible, then we can fit all items into our vehicles.\
Let's formulate bin packing problem for our case. Let be:\
$x_{ij}$ - represents if item $j$ goes to vehicle $i$\
$y_i$ - if the $i$ vehicle is used\
$v_j$ - volume of the item $j$\

$minimize \sum_{i=1}^{3}y_i$ \
$\sum_{j=1}^{7}v_jx_{ij} <= 3.6y_j$       $i$ = 1..$3$\
$\sum_{i=1}^{3}x_{ij} = 2$,                    $j$ = 1..7\
$x_{ij} \in \{0; 1\}$                       $i$ = 1..$3$, $j$ = 1..7\
$y_i \in \{0; 1\}$                        $i$ = 1..$3$

So the feasable solution for the task can be reached, if the optimizer finds a solution for this problem.\
To determine how to pack items into vehicles, parameter $x_{ij}$ shows to which vehicle each item is assigned 

In [13]:
T = [0.55 0.6 0.7 0.75 0.85 0.9 0.95]
model5 = Model(GLPK.Optimizer)
@variable(model5, x[1:7, 1:3], Bin)
@objective(model5, Min, sum(x[i, j] for i in 1:7, j in 1:3))
@constraint(model5, item_assignment[i=1:7], sum(x[i, j] for j in 1:3) == 2)
@constraint(model5, bin_capacity[j=1:3], sum(T[i] * x[i, j] for i in 1:7) <= 3.6)

3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 bin_capacity[1] : 0.55 x[1,1] + 0.6 x[2,1] + 0.7 x[3,1] + 0.75 x[4,1] + 0.85 x[5,1] + 0.9 x[6,1] + 0.95 x[7,1] ≤ 3.6
 bin_capacity[2] : 0.55 x[1,2] + 0.6 x[2,2] + 0.7 x[3,2] + 0.75 x[4,2] + 0.85 x[5,2] + 0.9 x[6,2] + 0.95 x[7,2] ≤ 3.6
 bin_capacity[3] : 0.55 x[1,3] + 0.6 x[2,3] + 0.7 x[3,3] + 0.75 x[4,3] + 0.85 x[5,3] + 0.9 x[6,3] + 0.95 x[7,3] ≤ 3.6

In [14]:
print(model5)

In [15]:
optimize!(model5)

In [16]:
@show termination_status(model5)
@show primal_status(model5)

termination_status(model5) = MathOptInterface.OPTIMAL
primal_status(model5) = MathOptInterface.FEASIBLE_POINT


FEASIBLE_POINT::ResultStatusCode = 1

Feasible solution is found, which means we can assign all items into 3 vehicles.\
Let's see how they are assigned

In [17]:
@show objective_value(model5)
@show value.(x)

objective_value(model5) = 14.0
value.(x) = [1.0 1.0 0.0; 1.0 1.0 0.0; 1.0 1.0 0.0; 0.0 1.0 1.0; 1.0 0.0 1.0; 1.0 0.0 1.0; 0.0 1.0 1.0]


7×3 Matrix{Float64}:
 1.0  1.0  0.0
 1.0  1.0  0.0
 1.0  1.0  0.0
 0.0  1.0  1.0
 1.0  0.0  1.0
 1.0  0.0  1.0
 0.0  1.0  1.0

## Answer
To the 1 vehicle go items: 1, 2, 3, 5, 6 (sum 3.6)\
To the 2 vehicle go items: 1, 2, 3, 4, 7 (sum 3.55)\
To the 3 vehicle go items: 4, 5, 6, 7 (sum 3.45)

# Exercise 4
Consider a public swimming pool. In the following table, we give a list of seven potential lifeguards. For each lifeguard, we have the time he/she wants to start and finish work and how much he/she wishes to be paid for the work. The problem is to find a selection of lifeguards so that there is at least one (but possibly more than one) lifeguard present at each time between 1pm and 9pm. 

|Lifeguards|Joy|Dean|Tim|Celicia|Beth|Ivar|Eilene|
|---|---|---|---|---|---|---|---|
|Hours|1-5|1-3|4-7|4-9|6-9|5-8|8-9|
|Amount required|30|18|21|38|20|22|9|

Formulate this problem as an IP.

Formulating an IP problem. Let $x_i$ be a note whether to select $i$ lifeguard or not.\
$c_i$ is the amount required by the $i$ lifeguard.\
Then the problem can be formulated as:\
$minimize \sum_{i}c_i*x_i$ - minimize the required amount for lifeguards\
Let matrix $a_{ij}$ be an indicator if lifeguard $i$ is available at time $j$ \
Then:\
$\sum_{i}a_{ij}*x_i >= 1, \forall j$ - at least one lifeguard presents at each time\
$x_{ij} \in \{0; 1\}$

In [58]:
hours = [
    1 1 1 1 1 0 0 0 0;
    1 1 1 0 0 0 0 0 0;
    0 0 0 1 1 1 1 0 0;
    0 0 0 1 1 1 1 1 1;
    0 0 0 0 0 1 1 1 1;
    0 0 0 0 1 1 1 1 0;
    0 0 0 0 0 0 0 1 1]
amount_required = [30, 18, 21, 38, 20, 22, 9]

7-element Vector{Int64}:
 30
 18
 21
 38
 20
 22
  9

In [59]:
model6 = Model(GLPK.Optimizer)
@variable(model6, x[1:length(amount_required)], Bin)
@objective(model6, Min, sum(amount_required[i] * x[i] for i in 1:length(amount_required)))
@constraint(model6, cs[j=1:9], sum(hours[i, j] * x[i] for i in 1:7) >= 1)

9-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cs[1] : x[1] + x[2] ≥ 1
 cs[2] : x[1] + x[2] ≥ 1
 cs[3] : x[1] + x[2] ≥ 1
 cs[4] : x[1] + x[3] + x[4] ≥ 1
 cs[5] : x[1] + x[3] + x[4] + x[6] ≥ 1
 cs[6] : x[3] + x[4] + x[5] + x[6] ≥ 1
 cs[7] : x[3] + x[4] + x[5] + x[6] ≥ 1
 cs[8] : x[4] + x[5] + x[6] + x[7] ≥ 1
 cs[9] : x[4] + x[5] + x[7] ≥ 1

In [60]:
print(model6)

In [61]:
optimize!(model6)

In [62]:
@show termination_status(model6)
@show primal_status(model6)

termination_status(model6) = MathOptInterface.OPTIMAL
primal_status(model6) = MathOptInterface.FEASIBLE_POINT


FEASIBLE_POINT::ResultStatusCode = 1

In [63]:
@show objective_value(model6)
@show value.(x)

objective_value(model6) = 48.0
value.(x) = [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0]


7-element Vector{Float64}:
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 1.0

## Answer
We need to hire Dean, Tim and Eilene.\
The total amount to be paid is 48.